In [1]:
from functools import partial

import numpy as np
import os

from math import ceil
from functools import partial

import tensorflow as tf

import librosa

C:\Users\chaow\Anaconda3\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\chaow\Anaconda3\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\chaow\Anaconda3\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\chaow\Anaconda3\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.

In [2]:
def listdirInMac(path):
    os_list = os.listdir(path)
    for item in os_list:
        if item.startswith('.') and os.path.isfile(os.path.join(path, item)):
            os_list.remove(item)
    return os_list

def traversalDir_FirstDir(path):
    dict = {}
    files = listdirInMac(path)
    for file in files:
        m = os.path.join(path, file)
        h = os.path.split(m)
        dict[h[1]] = []
        song_wav = listdirInMac(m)
        m = m + '/'
        for track in song_wav:
            value = os.path.join(m, track)
            dict[h[1]].append(value)
    return dict

mix_path = traversalDir_FirstDir('DSD100subset/Mixtures/Dev')

sou_path = traversalDir_FirstDir('DSD100subset/Sources/Dev')

all_path = mix_path.copy()
for key in all_path.keys():
    all_path[key].extend(sou_path[key])

print("%d mixtures in the given path to train"%len(mix_path))
print(all_path)

2 mixtures in the given path to train
{"055 - Angels In Amplifiers - I'm Alright": ["DSD100subset/Mixtures/Dev\\055 - Angels In Amplifiers - I'm Alright/mixture.wav", "DSD100subset/Sources/Dev\\055 - Angels In Amplifiers - I'm Alright/bass.wav", "DSD100subset/Sources/Dev\\055 - Angels In Amplifiers - I'm Alright/drums.wav", "DSD100subset/Sources/Dev\\055 - Angels In Amplifiers - I'm Alright/other.wav", "DSD100subset/Sources/Dev\\055 - Angels In Amplifiers - I'm Alright/vocals.wav"], '081 - Patrick Talbot - Set Me Free': ['DSD100subset/Mixtures/Dev\\081 - Patrick Talbot - Set Me Free/mixture.wav', 'DSD100subset/Sources/Dev\\081 - Patrick Talbot - Set Me Free/bass.wav', 'DSD100subset/Sources/Dev\\081 - Patrick Talbot - Set Me Free/drums.wav', 'DSD100subset/Sources/Dev\\081 - Patrick Talbot - Set Me Free/other.wav', 'DSD100subset/Sources/Dev\\081 - Patrick Talbot - Set Me Free/vocals.wav']}


In [3]:
def loadAudioFile(filePath):
    audio, sampleRate = librosa.load(filePath, sr = 44100)  # Load an audio file as a floating point time series， 默认采样率sr=22050
    return audio, sampleRate

# Return a 2d numpy array of the spectrogram
def audioFileToSpectrogram(audioFile, fftWindowSize = 1024):
    # time resolution N/fs=1024/44100=23ms
    spectrogram = librosa.stft(audioFile, fftWindowSize)   # 返回复数值矩阵D , STFT矩阵D中的行数是（1 + 第二个参数一般2的幂n_fft / 2）
    phase = np.angle(spectrogram)
    amplitude = np.log1p(np.abs(spectrogram))   # np.abs(D[f, t]) is the magnitude of frequency bin f at frame帧 t， loge(1+ )
    return amplitude, phase

频谱矩阵做random time stretch

In [4]:
import tensorflow as tf

# audio/spectrogram.py

def time_stretch(
        spectrograms,
        factor=1.0,
        method=tf.image.ResizeMethod.BILINEAR):
    """ Time stretch a spectrogram preserving shape in tensorflow. Note that
    this is an approximation in the frequency domain.
    :param spectrogram: Input spectrogram to be time stretched as tensor.
    :param factor: (Optional) Time stretch factor, must be >0, default to 1.
    :param mehtod: (Optional) Interpolation method, default to BILINEAR.
    :returns: Time stretched spectrogram as tensor with same shape.
    """
    F = tf.shape(spectrograms)[0]
    T = tf.shape(spectrograms)[1]
    T_ts = tf.cast(tf.cast(T, tf.float32) * factor, tf.int32)[0]  #cast: 改数据类型
    print(T, T_ts, F)  # 7587, 6097, 513
    ts_spec = tf.image.resize(
        spectrograms,
        size = [F, T_ts],
        method=method)      # 双线性插值
    return tf.image.resize_with_crop_or_pad(ts_spec, F, T)   # 补/切回原来形状


def random_time_stretch(spectrogram, factor_min=0.9, factor_max=1.1, **kwargs):
    """ Time stretch a spectrogram preserving shape with random ratio in
    tensorflow. Applies time_stretch to spectrogram with a random ratio drawn
    uniformly in [factor_min, factor_max].
    :param spectrogram: Input spectrogram to be time stretched as tensor.
    :param factor_min: (Optional) Min time stretch factor, default to 0.9.
    :param factor_max: (Optional) Max time stretch factor, default to 1.1.
    :returns: Randomly time stretched spectrogram as tensor with same shape.
    """
    factor = tf.random.uniform(
        shape=(1,),
        seed=0) * (factor_max - factor_min) + factor_min
    return time_stretch(spectrogram, factor=factor, **kwargs)


频谱矩阵做random pitch shift

In [5]:
def pitch_shift(
        spectrograms,
        semitone_shift=0.0,
        method=tf.image.ResizeMethod.BILINEAR):
    """ Pitch shift a spectrogram preserving shape in tensorflow. Note that
    this is an approximation in the frequency domain.
    :param spectrogram: Input spectrogram to be pitch shifted as tensor.
    :param semitone_shift: (Optional) Pitch shift in semitone半音, default to 0.0.
    :param mehtod: (Optional) Interpolation method, default to BILINEAR.
    :returns: Pitch shifted spectrogram (same shape as spectrogram).
    """
    factor = 2 ** (semitone_shift / 12.)
    F = tf.shape(spectrograms)[0]
    T = tf.shape(spectrograms)[1]
    F_ps = tf.cast(tf.cast(F, tf.float32) * factor, tf.int32)[0]
    ps_spec = tf.image.resize(
        spectrograms,
        [F_ps,T],
        method=method)
    paddings = [[0, tf.maximum(0, F - F_ps)], [0, 0], [0, 0]]
    return tf.pad(ps_spec[:F,:,:], paddings, 'CONSTANT')


def random_pitch_shift(spectrogram, shift_min=-1., shift_max=1., **kwargs):
    """ Pitch shift a spectrogram preserving shape with random ratio in
    tensorflow. Applies pitch_shift to spectrogram with a random shift
    amount (expressed in semitones) drawn uniformly in [shift_min, shift_max].
    :param spectrogram: Input spectrogram to be pitch shifted as tensor.
    :param shift_min: (Optional) Min pitch shift in semitone, default to -1.
    :param shift_max: (Optional) Max pitch shift in semitone, default to 1.
    :returns: Randomly pitch shifted spectrogram (same shape as spectrogram).
    """
    semitone_shift = tf.random.uniform(
        shape=(1,),
        seed=0) * (shift_max - shift_min) + shift_min
    return pitch_shift(spectrogram, semitone_shift=semitone_shift, **kwargs)

In [6]:
def expandToGrid(spectrogram, time_scale, ratio_overlap):
    newY = int(ceil((spectrogram.shape[1] - time_scale)/(1-ratio_overlap)/time_scale) * (1-ratio_overlap)*time_scale)+time_scale  # ceil取上
    #newX = int(spectrogram.shape[0]/64)*64
    newX = 512
    newSpectrogram = np.zeros((newX, newY))   # 右，下：多一些0，至能被girdsize整除
    print(spectrogram.shape,"original shape")
    print(newSpectrogram.shape,"expanded shape")
    newSpectrogram[:512, :spectrogram.shape[1]] = spectrogram[:512,:]
    return newSpectrogram

In [7]:
def chop(matrix, time_scale, ratio_overlap):
    slices = []
    for time in range(0, ceil((matrix.shape[1]-time_scale)/time_scale/(1-ratio_overlap))+1):   # 列
            s = matrix[: (int(matrix.shape[0]/64))*64,
                       int(time * time_scale*(1-ratio_overlap)) :int(time * time_scale*(1-ratio_overlap))+time_scale ]
            slices.append(s)
    print(s.shape)
    return slices

In [8]:
def song2spectrogram(all_path_para, fftWindowSize=1024, time_scale=64, overlap_ratio=0.5):
    

    x = np.empty((0,512, 64), float)
    x_phase = []
    y_bass = np.empty((0,512, 64), float)
    y_drums = np.empty((0,512, 64), float)
    y_other = np.empty((0,512, 64), float)
    y_vocals = np.empty((0,512, 64), float)
    # 第0维：每首歌
    # 第1维：height = f
    # 第2维：width = t
    # 第3维：每个音轨 - x,y_bass,y_drums,y_other,y_vocals 共5个
    for key in all_path_para:   # 每首歌
        path_list = all_path_para[key][:]
        audio, sampleRate = loadAudioFile(path_list[0])
        spectrogram, phase = audioFileToSpectrogram(audio, fftWindowSize)
        each_song = np.empty([int(spectrogram.shape[0]),int(spectrogram.shape[1]),0])
        for path in path_list:    # 每个音轨
            audio, sampleRate = loadAudioFile(path)           
            spectrogram, phase = audioFileToSpectrogram(audio, fftWindowSize)
            spectrogram = spectrogram[:,:,np.newaxis]            
            each_song = np.append(each_song, spectrogram, axis=-1)
        print('sample rate: ',sampleRate)
        print("spectrogram shape:",spectrogram.shape)
        print("each_song shape: ", each_song.shape)  # (5, 513, 7587)
        
        # 输入输出都做数据增强：time stretch，pitch swift
        ts_spectrogram = random_time_stretch(each_song)   
        print("Time stretch Spectrogram shape:",ts_spectrogram.shape)
        ps_spectrogram = random_pitch_shift(each_song)
        print("Pitch swift Spectrogram shape:",ps_spectrogram.shape)
                
        [hight, weidth, stem] = each_song.shape
        for spect in [each_song, ts_spectrogram, ps_spectrogram]:
            for i in range(stem):
                expandedSpectrogram = expandToGrid(spect[:,:,i],time_scale,overlap_ratio)
                slices = chop(expandedSpectrogram, time_scale,overlap_ratio)  
                if i==0:
                    x = np.append(x, slices,axis=0)
                    x_phase.append(phase)
                elif i==1:
                    y_bass = np.append(y_bass, slices,axis=0)
                elif i==2:
                    y_drums = np.append(y_drums, slices,axis=0)
                elif i==3:
                    y_other = np.append(y_other, slices,axis=0)
                else:   #i==4
                    y_vocals = np.append(y_vocals, slices,axis=0)
                
    output = [x,x_phase,y_bass,y_drums,y_other,y_vocals]
  
    return output

list_dataset = song2spectrogram(all_path_para=all_path)

sample rate:  44100
spectrogram shape: (513, 7587, 1)
each_song shape:  (513, 7587, 5)
tf.Tensor(7587, shape=(), dtype=int32) tf.Tensor(6981, shape=(), dtype=int32) tf.Tensor(513, shape=(), dtype=int32)
Time stretch Spectrogram shape: (513, 7587, 5)
Pitch swift Spectrogram shape: (513, 7587, 5)
(513, 7587) original shape
(512, 7616) expanded shape
(512, 64)
(513, 7587) original shape
(512, 7616) expanded shape
(512, 64)
(513, 7587) original shape
(512, 7616) expanded shape
(512, 64)
(513, 7587) original shape
(512, 7616) expanded shape
(512, 64)
(513, 7587) original shape
(512, 7616) expanded shape
(512, 64)
(513, 7587) original shape
(512, 7616) expanded shape
(512, 64)
(513, 7587) original shape
(512, 7616) expanded shape
(512, 64)
(513, 7587) original shape
(512, 7616) expanded shape
(512, 64)
(513, 7587) original shape
(512, 7616) expanded shape
(512, 64)
(513, 7587) original shape
(512, 7616) expanded shape
(512, 64)
(513, 7587) original shape
(512, 7616) expanded shape
(512, 64)


In [9]:
list_dataset[2].shape

(1389, 512, 64)

In [17]:
x = list_dataset[0]
y_bass = list_dataset[2]
y_drums = list_dataset[3]
y_other = list_dataset[4]
y_vocals = list_dataset[5]

x = np.array(x)[:, :, :, np.newaxis]
y_bass = np.array(y_bass)[:, :, :, np.newaxis]
y_drums = np.array(y_drums)[:, :, :, np.newaxis]
y_other = np.array(y_drums)[:, :, :, np.newaxis]
y_vocals = np.array(y_drums)[:, :, :, np.newaxis]

print(x.shape, y_vocals.shape)

(1389, 512, 64, 1) (1389, 512, 64, 1, 1)


In [11]:
import numpy as np
import os
from tensorflow.keras.layers import Input, Reshape, Lambda
from tensorflow.keras.models import Model
from math import ceil
from tensorflow.keras.layers import (
    BatchNormalization,
    Concatenate,
    Conv2D,
    Conv2DTranspose,
    Dropout,
    multiply,
    ReLU)
from functools import partial

from tensorflow.keras.backend import int_shape, squeeze

import tensorflow as tf
from functools import partial

from tensorflow.keras.initializers import he_uniform

import numpy as np
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, BatchNormalization, UpSampling2D, Concatenate
from tensorflow.keras.metrics import Accuracy

In [12]:
def apply_unet():
    """ Apply a convolutionnal U-net to model a single instrument (one U-net
    is used for each instrument).
    :param input_tensor:
    :param output_name: (Optional) , default to 'output'
    :param params: (Optional) , default to empty dict.
    :param output_mask_logit: (Optional) , default to False.
    """
    conv_n_filters = [16, 32, 64, 128, 256, 512]
    conv_activation_layer = ReLU()
    deconv_activation_layer = ReLU()
    kernel_initializer = he_uniform(seed=50)
    conv2d_factory = partial(
        Conv2D,
        strides=(2, 2),
        padding='same',
        kernel_initializer=kernel_initializer)

    #input_tensor = Input(shape=(None, None, 1), name='input')
    input_tensor = Input(shape=(512, 64, 1), name='input')

    # First layer.
    conv1 = conv2d_factory(conv_n_filters[0], (5, 5))(input_tensor)
    batch1 = BatchNormalization(axis=-1)(conv1)
    rel1 = conv_activation_layer(batch1)
    # Second layer.
    conv2 = conv2d_factory(conv_n_filters[1], (5, 5))(rel1)
    batch2 = BatchNormalization(axis=-1)(conv2)
    rel2 = conv_activation_layer(batch2)
    # Third layer.
    conv3 = conv2d_factory(conv_n_filters[2], (5, 5))(rel2)
    batch3 = BatchNormalization(axis=-1)(conv3)
    rel3 = conv_activation_layer(batch3)
    # Fourth layer.
    conv4 = conv2d_factory(conv_n_filters[3], (5, 5))(rel3)
    batch4 = BatchNormalization(axis=-1)(conv4)
    rel4 = conv_activation_layer(batch4)
    # Fifth layer.
    conv5 = conv2d_factory(conv_n_filters[4], (5, 5))(rel4)
    batch5 = BatchNormalization(axis=-1)(conv5)
    rel5 = conv_activation_layer(batch5)
    # Sixth layer
    conv6 = conv2d_factory(conv_n_filters[5], (5, 5))(rel5)
    batch6 = BatchNormalization(axis=-1)(conv6)
    _ = conv_activation_layer(batch6)
    #
    #
    conv2d_transpose_factory = partial(
        Conv2DTranspose,
        strides=(2, 2),
        padding='same',
        kernel_initializer=kernel_initializer)
    #
    up1 = conv2d_transpose_factory(conv_n_filters[4], (5, 5))((conv6))
    up1 = deconv_activation_layer(up1)
    batch7 = BatchNormalization(axis=-1)(up1)
    drop1 = Dropout(0.5)(batch7)
    merge1 = Concatenate(axis=-1)([conv5, drop1])
    #
    up2 = conv2d_transpose_factory(conv_n_filters[3], (5, 5))((merge1))
    up2 = deconv_activation_layer(up2)
    batch8 = BatchNormalization(axis=-1)(up2)
    drop2 = Dropout(0.5)(batch8)
    merge2 = Concatenate(axis=-1)([conv4, drop2])
    #
    up3 = conv2d_transpose_factory(conv_n_filters[2], (5, 5))((merge2))
    up3 = deconv_activation_layer(up3)
    batch9 = BatchNormalization(axis=-1)(up3)
    drop3 = Dropout(0.5)(batch9)
    merge3 = Concatenate(axis=-1)([conv3, drop3])
    #
    up4 = conv2d_transpose_factory(conv_n_filters[1], (5, 5))((merge3))
    up4 = deconv_activation_layer(up4)
    batch10 = BatchNormalization(axis=-1)(up4)
    merge4 = Concatenate(axis=-1)([conv2, batch10])
    #
    up5 = conv2d_transpose_factory(conv_n_filters[0], (5, 5))((merge4))
    up5 = deconv_activation_layer(up5)
    batch11 = BatchNormalization(axis=-1)(up5)
    merge5 = Concatenate(axis=-1)([conv1, batch11])
    #
    up6 = conv2d_transpose_factory(1, (5, 5), strides=(2, 2))((merge5))
    up6 = deconv_activation_layer(up6)
    batch12 = BatchNormalization(axis=-1)(up6)
    # Last layer to ensure initial shape reconstruction.
    up7 = Conv2D(1, (4, 4), dilation_rate=(2, 2), activation='sigmoid', padding='same', kernel_initializer=kernel_initializer)((batch12))
    output = multiply([up7, input_tensor])

    m = Model(inputs=input_tensor, outputs=output)

    return m

In [13]:
m_unet = apply_unet()

In [14]:
m_unet.compile(loss='mean_squared_error', optimizer='adam', metrics=[Accuracy()])

In [15]:
m_unet.fit(x,y , batch_size=50, epochs=1, validation_split=0.1)

Train on 1250 samples, validate on 139 samples
1250/1250 [==============================] - 86s 69ms/sample - loss: 0.0444 - accuracy: 0.0289 - val_loss: 0.0363 - val_accuracy: 0.0022


In [ ]:
def softmax_unet(input_tensor, instruments, params={}):
    """ Apply softmax to multitrack unet in order to have mask suming to one.
    :param input_tensor: Tensor to apply blstm to.
    :param instruments: Iterable that provides a collection of instruments.
    :param params: (Optional) dict of BLSTM parameters.
    :returns: Created output tensor dict.
    """
    logit_mask_list = []
    for instrument in instruments:
        out_name = f'{instrument}_spectrogram'
        logit_mask_list.append(
            apply_unet(
                input_tensor,
                output_name=out_name,
                params=params,
                output_mask_logit=True))
    masks = Softmax(axis=4)(tf.stack(logit_mask_list, axis=4))
    output_dict = {}
    for i, instrument in enumerate(instruments):
        out_name = f'{instrument}_spectrogram'
        output_dict[out_name] = Multiply(name=out_name)([
            masks[..., i],
            input_tensor])
    return output_dict

In [ ]:
def _create_estimator(params):
    """ Creates estimator.
    :param params: TF params to build estimator from.
    :returns: Built estimator.
    """
    session_config = tf.compat.v1.ConfigProto()
    session_config.gpu_options.per_process_gpu_memory_fraction = 0.45
    estimator = tf.estimator.Estimator(
        model_fn=model_fn,
        model_dir=params['model_dir'],
        params=params,
        config=tf.estimator.RunConfig(
            save_checkpoints_steps=params['save_checkpoints_steps'],
            tf_random_seed=params['random_seed'],
            save_summary_steps=params['save_summary_steps'],
            session_config=session_config,
            log_step_count_steps=10,
            keep_checkpoint_max=2))
    return estimator

In [ ]:
def _create_train_spec(params, audio_adapter, audio_path):
    """ Creates train spec.
    :param params: TF params to build spec from.
    :returns: Built train spec.
    """
    input_fn = partial(get_training_dataset, params, audio_adapter, audio_path)
    train_spec = tf.estimator.TrainSpec(
        input_fn=input_fn,
        max_steps=params['train_max_steps'])
    return train_spec